In [1]:
import pandas as pd
import numpy as np
import json
import warnings
import spacy
warnings.filterwarnings("ignore")
import random

import nltk
from nltk.stem import WordNetLemmatizer

import re

from spacy.tokens import DocBin

from tqdm import tqdm

In [2]:
train_data = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\train.csv")
features = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\features.csv")
patient_notes = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\patient_notes.csv")

In [3]:
new_train = train_data[(train_data.feature_num == 6)][['pn_num','annotation',"location","feature_num"]]
new_train

,pn_num,annotation,location,feature_num
6,16,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']",6
19,41,[],[],6
32,46,"['adderall', 'Adderall']","['380 388', '730 738']",6
45,82,['aderall'],['389 396'],6
58,100,['adderall'],['595 603'],6
...,...,...,...,...
1241,2315,['Adderall'],['382 390'],6
1254,2425,[],[],6
1267,2428,['adderall'],['456 464'],6
1280,2430,['adderall'],['435 443'],6


In [4]:
new_train = new_train[new_train.annotation != '[]']
new_train

,pn_num,annotation,location,feature_num
6,16,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']",6
32,46,"['adderall', 'Adderall']","['380 388', '730 738']",6
45,82,['aderall'],['389 396'],6
58,100,['adderall'],['595 603'],6
84,188,"['aderol', 'aderol', 'aderol']","['457 463', '507 513', '655 661']",6
...,...,...,...,...
1228,2308,['adderall'],['584 592'],6
1241,2315,['Adderall'],['382 390'],6
1267,2428,['adderall'],['456 464'],6
1280,2430,['adderall'],['435 443'],6


In [5]:
new_train["annotation"] = new_train.annotation.str.replace('[','')
new_train["annotation"] = new_train.annotation.str.replace(']','')
new_train["location"] = new_train.location.str.replace('[','')
new_train["location"] = new_train.location.str.replace(']','')

new_train

,pn_num,annotation,location,feature_num
6,16,"'adderall', 'adderrall', 'adderrall'","'321 329', '404 413', '652 661'",6
32,46,"'adderall', 'Adderall'","'380 388', '730 738'",6
45,82,'aderall','389 396',6
58,100,'adderall','595 603',6
84,188,"'aderol', 'aderol', 'aderol'","'457 463', '507 513', '655 661'",6
...,...,...,...,...
1228,2308,'adderall','584 592',6
1241,2315,'Adderall','382 390',6
1267,2428,'adderall','456 464',6
1280,2430,'adderall','435 443',6


In [6]:
new_train = (new_train.set_index(['pn_num',"feature_num"]) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(new_train.columns, axis=1))
new_train

,pn_num,annotation,location,feature_num
0,16,'adderall','321 329',6
1,16,'adderrall','404 413',6
2,16,'adderrall','652 661',6
3,46,'adderall','380 388',6
4,46,'Adderall','730 738',6
...,...,...,...,...
130,2308,'adderall','584 592',6
131,2315,'Adderall','382 390',6
132,2428,'adderall','456 464',6
133,2430,'adderall','435 443',6


In [7]:
new_train["start_location"] = new_train["location"].apply(lambda x: x.split()[0][1:])
new_train["end_location"] = new_train["location"].apply(lambda x: x.split()[-1][:-1])
new_train

,pn_num,annotation,location,feature_num,start_location,end_location
0,16,'adderall','321 329',6,321,329
1,16,'adderrall','404 413',6,404,413
2,16,'adderrall','652 661',6,652,661
3,46,'adderall','380 388',6,380,388
4,46,'Adderall','730 738',6,730,738
...,...,...,...,...,...,...
130,2308,'adderall','584 592',6,584,592
131,2315,'Adderall','382 390',6,382,390
132,2428,'adderall','456 464',6,456,464
133,2430,'adderall','435 443',6,435,443


In [8]:
new_train = new_train.merge(patient_notes, on='pn_num', how='left')
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history
0,16,'adderall','321 329',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...
1,16,'adderrall','404 413',6,404,413,0,HPI: 17yo M presents with palpitations. Patien...
2,16,'adderrall','652 661',6,652,661,0,HPI: 17yo M presents with palpitations. Patien...
3,46,'adderall','380 388',6,380,388,0,Mr. Cleveland is a 17yo M who was consented by...
4,46,'Adderall','730 738',6,730,738,0,Mr. Cleveland is a 17yo M who was consented by...
...,...,...,...,...,...,...,...,...
130,2308,'adderall','584 592',6,584,592,0,17 yo male with no significant PMH presenting ...
131,2315,'Adderall','382 390',6,382,390,0,17 yo M who presents with episodes of heart pa...
132,2428,'adderall','456 464',6,456,464,0,dillon cleveland is a 17yo male with no PMH th...
133,2430,'adderall','435 443',6,435,443,0,Mr. Cleveland is a 17 yo male who presents wit...


In [9]:
new_train["New_annotation"] = new_train.apply(lambda x: x.pn_history[int(x.start_location):int(x.end_location)],axis=1)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation
0,16,'adderall','321 329',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...,adderall
1,16,'adderrall','404 413',6,404,413,0,HPI: 17yo M presents with palpitations. Patien...,adderrall
2,16,'adderrall','652 661',6,652,661,0,HPI: 17yo M presents with palpitations. Patien...,adderrall
3,46,'adderall','380 388',6,380,388,0,Mr. Cleveland is a 17yo M who was consented by...,adderall
4,46,'Adderall','730 738',6,730,738,0,Mr. Cleveland is a 17yo M who was consented by...,Adderall
...,...,...,...,...,...,...,...,...,...
130,2308,'adderall','584 592',6,584,592,0,17 yo male with no significant PMH presenting ...,adderall
131,2315,'Adderall','382 390',6,382,390,0,17 yo M who presents with episodes of heart pa...,Adderall
132,2428,'adderall','456 464',6,456,464,0,dillon cleveland is a 17yo male with no PMH th...,adderall
133,2430,'adderall','435 443',6,435,443,0,Mr. Cleveland is a 17 yo male who presents wit...,adderall


In [10]:
new_train["New_annotation"] = new_train["New_annotation"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))
new_train["New_annotation"] = new_train["New_annotation"].apply(lambda x: re.sub(r'''[']''','',x))
new_train["New_annotation"] = new_train["New_annotation"].apply(lambda x: x.strip())
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation
0,16,'adderall','321 329',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...,adderall
1,16,'adderrall','404 413',6,404,413,0,HPI: 17yo M presents with palpitations. Patien...,adderrall
2,16,'adderrall','652 661',6,652,661,0,HPI: 17yo M presents with palpitations. Patien...,adderrall
3,46,'adderall','380 388',6,380,388,0,Mr. Cleveland is a 17yo M who was consented by...,adderall
4,46,'Adderall','730 738',6,730,738,0,Mr. Cleveland is a 17yo M who was consented by...,Adderall
...,...,...,...,...,...,...,...,...,...
130,2308,'adderall','584 592',6,584,592,0,17 yo male with no significant PMH presenting ...,adderall
131,2315,'Adderall','382 390',6,382,390,0,17 yo M who presents with episodes of heart pa...,Adderall
132,2428,'adderall','456 464',6,456,464,0,dillon cleveland is a 17yo male with no PMH th...,adderall
133,2430,'adderall','435 443',6,435,443,0,Mr. Cleveland is a 17 yo male who presents wit...,adderall


In [11]:
new_train["New_pn_history"] = new_train["pn_history"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))
new_train["New_pn_history"] = new_train["New_pn_history"].apply(lambda x: re.sub(r'''[']''','',x))
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history
0,16,'adderall','321 329',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...
1,16,'adderrall','404 413',6,404,413,0,HPI: 17yo M presents with palpitations. Patien...,adderrall,HPI: 17yo M presents with palpitations. Patien...
2,16,'adderrall','652 661',6,652,661,0,HPI: 17yo M presents with palpitations. Patien...,adderrall,HPI: 17yo M presents with palpitations. Patien...
3,46,'adderall','380 388',6,380,388,0,Mr. Cleveland is a 17yo M who was consented by...,adderall,Mr. Cleveland is a 17yo M who was consented by...
4,46,'Adderall','730 738',6,730,738,0,Mr. Cleveland is a 17yo M who was consented by...,Adderall,Mr. Cleveland is a 17yo M who was consented by...
...,...,...,...,...,...,...,...,...,...,...
130,2308,'adderall','584 592',6,584,592,0,17 yo male with no significant PMH presenting ...,adderall,17 yo male with no significant PMH presenting ...
131,2315,'Adderall','382 390',6,382,390,0,17 yo M who presents with episodes of heart pa...,Adderall,17 yo M who presents with episodes of heart pa...
132,2428,'adderall','456 464',6,456,464,0,dillon cleveland is a 17yo male with no PMH th...,adderall,dillon cleveland is a 17yo male with no PMH th...
133,2430,'adderall','435 443',6,435,443,0,Mr. Cleveland is a 17 yo male who presents wit...,adderall,Mr. Cleveland is a 17 yo male who presents wit...


In [12]:
from spellchecker import SpellChecker

spell = SpellChecker()

def correct_word_spelling(word):
    
    misspelled = spell.unknown(word)
    
    result = spell.correction(word)
    
    return(result)

In [13]:
new_train["New_pn_history"] = new_train.apply(lambda x : x.New_pn_history.replace(x.pn_history[int(x.start_location):int(x.end_location)], spell.correction(x.pn_history[int(x.start_location):int(x.end_location)])), axis = 1)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history
0,16,'adderall','321 329',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...
1,16,'adderrall','404 413',6,404,413,0,HPI: 17yo M presents with palpitations. Patien...,adderrall,HPI: 17yo M presents with palpitations. Patien...
2,16,'adderrall','652 661',6,652,661,0,HPI: 17yo M presents with palpitations. Patien...,adderrall,HPI: 17yo M presents with palpitations. Patien...
3,46,'adderall','380 388',6,380,388,0,Mr. Cleveland is a 17yo M who was consented by...,adderall,Mr. Cleveland is a 17yo M who was consented by...
4,46,'Adderall','730 738',6,730,738,0,Mr. Cleveland is a 17yo M who was consented by...,Adderall,Mr. Cleveland is a 17yo M who was consented by...
...,...,...,...,...,...,...,...,...,...,...
130,2308,'adderall','584 592',6,584,592,0,17 yo male with no significant PMH presenting ...,adderall,17 yo male with no significant PMH presenting ...
131,2315,'Adderall','382 390',6,382,390,0,17 yo M who presents with episodes of heart pa...,Adderall,17 yo M who presents with episodes of heart pa...
132,2428,'adderall','456 464',6,456,464,0,dillon cleveland is a 17yo male with no PMH th...,adderall,dillon cleveland is a 17yo male with no PMH th...
133,2430,'adderall','435 443',6,435,443,0,Mr. Cleveland is a 17 yo male who presents wit...,adderall,Mr. Cleveland is a 17 yo male who presents wit...


In [14]:
new_train["New_annotation"] = new_train["New_annotation"].apply(correct_word_spelling)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history
0,16,'adderall','321 329',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...
1,16,'adderrall','404 413',6,404,413,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...
2,16,'adderrall','652 661',6,652,661,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...
3,46,'adderall','380 388',6,380,388,0,Mr. Cleveland is a 17yo M who was consented by...,adderall,Mr. Cleveland is a 17yo M who was consented by...
4,46,'Adderall','730 738',6,730,738,0,Mr. Cleveland is a 17yo M who was consented by...,Adderall,Mr. Cleveland is a 17yo M who was consented by...
...,...,...,...,...,...,...,...,...,...,...
130,2308,'adderall','584 592',6,584,592,0,17 yo male with no significant PMH presenting ...,adderall,17 yo male with no significant PMH presenting ...
131,2315,'Adderall','382 390',6,382,390,0,17 yo M who presents with episodes of heart pa...,Adderall,17 yo M who presents with episodes of heart pa...
132,2428,'adderall','456 464',6,456,464,0,dillon cleveland is a 17yo male with no PMH th...,adderall,dillon cleveland is a 17yo male with no PMH th...
133,2430,'adderall','435 443',6,435,443,0,Mr. Cleveland is a 17 yo male who presents wit...,adderall,Mr. Cleveland is a 17 yo male who presents wit...


In [15]:
new_train["new_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history),axis =1)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,'adderall','321 329',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...,"<re.Match object; span=(321, 329), match='adde..."
1,16,'adderrall','404 413',6,404,413,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...,"<re.Match object; span=(321, 329), match='adde..."
2,16,'adderrall','652 661',6,652,661,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...,"<re.Match object; span=(321, 329), match='adde..."
3,46,'adderall','380 388',6,380,388,0,Mr. Cleveland is a 17yo M who was consented by...,adderall,Mr. Cleveland is a 17yo M who was consented by...,"<re.Match object; span=(380, 388), match='adde..."
4,46,'Adderall','730 738',6,730,738,0,Mr. Cleveland is a 17yo M who was consented by...,Adderall,Mr. Cleveland is a 17yo M who was consented by...,"<re.Match object; span=(730, 738), match='Adde..."
...,...,...,...,...,...,...,...,...,...,...,...
130,2308,'adderall','584 592',6,584,592,0,17 yo male with no significant PMH presenting ...,adderall,17 yo male with no significant PMH presenting ...,"<re.Match object; span=(581, 589), match='adde..."
131,2315,'Adderall','382 390',6,382,390,0,17 yo M who presents with episodes of heart pa...,Adderall,17 yo M who presents with episodes of heart pa...,"<re.Match object; span=(381, 389), match='Adde..."
132,2428,'adderall','456 464',6,456,464,0,dillon cleveland is a 17yo male with no PMH th...,adderall,dillon cleveland is a 17yo male with no PMH th...,"<re.Match object; span=(456, 464), match='adde..."
133,2430,'adderall','435 443',6,435,443,0,Mr. Cleveland is a 17 yo male who presents wit...,adderall,Mr. Cleveland is a 17 yo male who presents wit...,"<re.Match object; span=(434, 442), match='adde..."


In [16]:
new_train["start_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history).start(),axis =1)
new_train["end_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history).end(),axis =1)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,'adderall','321 329',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...,"<re.Match object; span=(321, 329), match='adde..."
1,16,'adderrall','404 413',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...,"<re.Match object; span=(321, 329), match='adde..."
2,16,'adderrall','652 661',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...,"<re.Match object; span=(321, 329), match='adde..."
3,46,'adderall','380 388',6,380,388,0,Mr. Cleveland is a 17yo M who was consented by...,adderall,Mr. Cleveland is a 17yo M who was consented by...,"<re.Match object; span=(380, 388), match='adde..."
4,46,'Adderall','730 738',6,730,738,0,Mr. Cleveland is a 17yo M who was consented by...,Adderall,Mr. Cleveland is a 17yo M who was consented by...,"<re.Match object; span=(730, 738), match='Adde..."
...,...,...,...,...,...,...,...,...,...,...,...
130,2308,'adderall','584 592',6,581,589,0,17 yo male with no significant PMH presenting ...,adderall,17 yo male with no significant PMH presenting ...,"<re.Match object; span=(581, 589), match='adde..."
131,2315,'Adderall','382 390',6,381,389,0,17 yo M who presents with episodes of heart pa...,Adderall,17 yo M who presents with episodes of heart pa...,"<re.Match object; span=(381, 389), match='Adde..."
132,2428,'adderall','456 464',6,456,464,0,dillon cleveland is a 17yo male with no PMH th...,adderall,dillon cleveland is a 17yo male with no PMH th...,"<re.Match object; span=(456, 464), match='adde..."
133,2430,'adderall','435 443',6,434,442,0,Mr. Cleveland is a 17 yo male who presents wit...,adderall,Mr. Cleveland is a 17 yo male who presents wit...,"<re.Match object; span=(434, 442), match='adde..."


In [17]:
sep_combined_case0  = new_train.copy()

In [18]:
row_index = []
start_location = []
end_location = []
pn_notes = set()
drop_index = []
for index,row in sep_combined_case0.iterrows():   # Iterating every row in trainn data
    if row["pn_num"] not in pn_notes:    # Verifying whether there is a change in pn_num to reinitialize start and end location lists
        start_location = []
        end_location = []
        pn_notes.add(row["pn_num"])      # Appending pn_num
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
    else: 
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
            
            if row["start_location"] in start_location:
                if row["end_location"] >= sep_combined_case0.iloc[index -1 ,5]:
                    drop_index.append(index -1)
                else:
                    drop_index.append(index)
            elif row["end_location"] in end_location:
                if row["start_location"] <= sep_combined_case0.iloc[index -1,4]:
                    drop_index.append(index - 1)
                else:
                    drop_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])

In [19]:
print(drop_index), len(drop_index)

[0, 1, 7, 8, 25, 26, 30, 33, 33, 37, 39, 40, 45, 54, 60, 61, 66, 68, 83, 85, 93, 98, 107, 109, 126]


(None, 25)

In [20]:
sep_combined_case0 = sep_combined_case0.drop(index=drop_index,axis = 0)
sep_combined_case0

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
2,16,'adderrall','652 661',6,321,329,0,HPI: 17yo M presents with palpitations. Patien...,adderall,HPI: 17yo M presents with palpitations. Patien...,"<re.Match object; span=(321, 329), match='adde..."
3,46,'adderall','380 388',6,380,388,0,Mr. Cleveland is a 17yo M who was consented by...,adderall,Mr. Cleveland is a 17yo M who was consented by...,"<re.Match object; span=(380, 388), match='adde..."
4,46,'Adderall','730 738',6,730,738,0,Mr. Cleveland is a 17yo M who was consented by...,Adderall,Mr. Cleveland is a 17yo M who was consented by...,"<re.Match object; span=(730, 738), match='Adde..."
5,82,'aderall','389 396',6,389,397,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,adderall,17 yo M w no cardiac or arrhythmia PMH presen...,"<re.Match object; span=(389, 397), match='adde..."
6,100,'adderall','595 603',6,594,602,0,HPI: Dillon Cleveland is an otherwise healthy ...,adderall,HPI: Dillon Cleveland is an otherwise healthy ...,"<re.Match object; span=(594, 602), match='adde..."
...,...,...,...,...,...,...,...,...,...,...,...
130,2308,'adderall','584 592',6,581,589,0,17 yo male with no significant PMH presenting ...,adderall,17 yo male with no significant PMH presenting ...,"<re.Match object; span=(581, 589), match='adde..."
131,2315,'Adderall','382 390',6,381,389,0,17 yo M who presents with episodes of heart pa...,Adderall,17 yo M who presents with episodes of heart pa...,"<re.Match object; span=(381, 389), match='Adde..."
132,2428,'adderall','456 464',6,456,464,0,dillon cleveland is a 17yo male with no PMH th...,adderall,dillon cleveland is a 17yo male with no PMH th...,"<re.Match object; span=(456, 464), match='adde..."
133,2430,'adderall','435 443',6,434,442,0,Mr. Cleveland is a 17 yo male who presents wit...,adderall,Mr. Cleveland is a 17 yo male who presents wit...,"<re.Match object; span=(434, 442), match='adde..."


In [21]:
sep_combined_case0 = sep_combined_case0.drop(index=19,axis = 0)

In [22]:
sep_combined_case0 = sep_combined_case0.drop(index=34,axis = 0)

In [23]:
pn_Num_list = list(sep_combined_case0.pn_num.unique())

In [24]:
final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in sep_combined_case0[sep_combined_case0.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = sep_combined_case0[sep_combined_case0.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list

In [25]:
final_train[5]

('HPI: Patient is a 17 yo m with a c o of palpitations.  Palpitations began a few months ago. States that palpitations are sudden  unpredictable and feel like his heart is pounding fast jumping out of his chest. Typically these episodes last 3-4 minutes and resolve on their own. His most recent episode was 2 days ago and lasted about 10 minutes. During this epsiode he felt lightheaded  Short of breath and had chest pressure located in the middle of his chest. Denies any sweating  changes in hair or bowel movements.  ROS: Negative except as stated above  PMH: None  Meds: Takes his roommates Adderall to help study  Allergies: NKDA  PSHx: None  FH: Mother has a thyroid problem  Father had a MI this past year at age 53  SH: denies to',
 {'entities': [(595, 603, 'Feature_6')]})

In [26]:
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [27]:
# Opening JSON file
f = open("sample.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [28]:
nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [29]:
train = data[:int(len(data)*0.8)]
validation = data[int(len(data)*0.8):]

len(train),len(validation)

(72, 19)

In [30]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 640.09it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [31]:
# Validation data set
camp_train = create_training(validation)
camp_train.to_disk("nbme_validation.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 760.00it/s]


In [32]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-04-05 23:55:05.919538: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-05 23:55:05.919599: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [33]:
!python -m spacy train config.cfg --output ./output

[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     68.67    0.00    0.00    0.00    0.00
  2     200         37.80   1324.38   60.87   58.33   63.64    0.61
  5     400       1613.96    297.22   56.41   64.71   50.00    0.56
  8     600       1475.00    219.00   53.66   57.89   50.00    0.54
 11     800        335.76    112.20   65.00   72.22   59.09    0.65
 13    1000         57.24     50.76   58.54   63.16   54.55    0.59
 16    1200         35.89     23.06   60.47   61.90   59.09    0.60
 19    1400         51.69     34.49   59.46   73.33 

2022-04-05 23:55:09.687254: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-05 23:55:09.687295: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-04-05 23:55:12,935] [INFO] Set up nlp object from config
[2022-04-05 23:55:12,943] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-04-05 23:55:12,946] [INFO] Created vocabulary
[2022-04-05 23:55:12,949] [INFO] Finished initializing nlp object
[2022-04-05 23:55:13,597] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
